### REPTE 3: Histologia digital - Mar Blazquez

In [8]:
import pandas as pd
import numpy as np
import cv2
import os
import glob
import random
import os
import glob
import random
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data
import torchvision.transforms as transforms
import math
import itertools
from torch import nn as nn
import torch.nn.functional as F
from torch import Tensor
from numpy.matlib import repmat
import torch.optim as optim
from collections import OrderedDict
import cv2
import wandb
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
# save dicts to csv
import csv

In [9]:
import cv2
import numpy as np
from sklearn.metrics import roc_curve, auc

def get_fred(model, dataloader,device):
    model.eval()  # Cambiar el modelo a modo evaluación
    fred_values = []  # Almacenar los valores de F_red

    if device == 'cuda' and torch.cuda.is_available():
        device = 'cuda'
    else:
        device = 'cpu'
    j=0

    with torch.no_grad():
        for i,images in enumerate(dataloader):
            if type(images) == list:
                # images to tensor
                image = images[0]
                image = image.to(device)
                image = image/255.0
            else:
                # Enviar imágenes al dispositivo
                images = images.to(device)
                image = images/255.0

            # Obtener las reconstrucciones del modelo
            reconstructed = model(image)

            # permute HWC
            image = image[0]
            image = image.permute(1, 2, 0).cpu().numpy()
            reconstructed = reconstructed[0]
            reconstructed = reconstructed.permute(1, 2, 0).cpu().numpy()

            # Convertir a formato HUE (HSV)
            hsv_orig = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            hsv_rec = cv2.cvtColor(reconstructed, cv2.COLOR_RGB2HSV)

            # Calcular F_red para la imagen original y la reconstrucción
            h_orig = hsv_orig[:, :, 0]
            h_rec = hsv_rec[:, :, 0]

            mask_orig = ((h_orig >= 160) & (h_orig <= 179)) | (h_orig >= 0) & (h_orig <= 20)
            mask_rec = ((h_rec >= 160) & (h_rec <= 179)) | (h_rec >= 0) & (h_rec <= 20)

            fred = np.sum(mask_orig) / max(np.sum(mask_rec), 1)

            # Almacenar valores y etiquetas reales
            fred_values.append(fred)

    return fred_values

# Calcular F_red para las imágenes de densidad 0
#fred_values = get_fred(model, dataloader_annotated, device="cpu")
#print('len fred',len(fred_values))

In [10]:
def load_holdout_dataset(data_dir, transform=None):
    """
    Carga el dataset de holdout desde un directorio especificado.

    Args:
        data_dir (str): Directorio con los datos del conjunto holdout.
        transform (torchvision.transforms.Compose): Transformaciones para aplicar a las imágenes.

    Returns:
        Dataset: Dataset PyTorch listo para usarse.
    """
    if transform is None:
        transform = transforms.Compose([
            transforms.Resize((224, 224)),  # Ajustar a la entrada del modelo
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    holdout_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
    return holdout_dataset

def plot_confusion_matrix(cm, class_names):
    """
    Genera un gráfico de matriz de confusión.

    Args:
        cm (ndarray): Matriz de confusión.
        class_names (list): Nombres de las clases.
    """
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.ylabel('Etiqueta Real')
    plt.xlabel('Etiqueta Predicha')
    plt.title('Matriz de Confusión')
    plt.show()

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F


def init_weights_transformer(module,InitParams):
    Type=InitParams['Type']
    U=InitParams['U']

    if Type=='ori':
        init_weights_transformer_ori(module)
    elif Type=='all':
        init_weights_transformer_all(module,U)
    else:
        init_weights_transformer_exceptnorm(module,U)

def init_weights_transformer_all(module,U=[0,1]):
    param=module.state_dict()

    for name in param.keys():
        if name.find('norm')<0:
           nn.init.uniform_(param[name],a=U[0],b=U[1])


def init_weights_transformer_exceptnorm(module,U=[0,1]):
    param=module.state_dict()

    for name in param.keys():
        if name.find('norm')<0:
           if (name.find('weight')>0):
                nn.init.uniform_(param[name],a=U[0],b=U[1])
           elif (name.find('bias')>0):
                nn.init.constant_(param[name], 0)
    #
def init_weights_transformer_ori(module):
    initrange = 0.1
    nn.init.uniform_(module.encoder.weight, -initrange, initrange)
    nn.init.zeros_(module.decoder1[0].bias)
    nn.init.zeros_(module.decoder2.bias)
    nn.init.uniform_(module.decoder1[0].weight, -initrange, initrange)
    nn.init.uniform_(module.decoder2.weight, -initrange, initrange)


def init_weights_xavier_normal(module):
    for m in module.modules():
        if isinstance(m, nn.Conv1d):
            nn.init.xavier_normal_(m.weight)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Conv2d):
            nn.init.xavier_normal_(m.weight)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Conv3d):
            nn.init.xavier_normal_(m.weight)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm1d):
            nn.init.normal_(m.weight, 0, 0.01)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm2d):
            nn.init.normal_(m.weight, 0, 0.01)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm3d):
            nn.init.normal_(m.weight, 0, 0.01)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LSTM):
            for param in m.parameters():
                if len(param.shape) >= 2:
                    nn.init.orthogonal_(param.data)
                else:
                    nn.init.normal_(param.data)

def init_weights_xavier_uniform(module):
    for m in module.modules():
        if isinstance(m, nn.Conv1d):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Conv2d):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Conv3d):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm1d):
            nn.init.uniform_(m.weight, a=0, b=1)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm2d):
            nn.init.uniform_(m.weight, a=0, b=1)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm3d):
            nn.init.uniform_(m.weight, a=0, b=1)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LSTM):
            for param in m.parameters():
                if len(param.shape) >= 2:
                    nn.init.orthogonal_(param.data)
                else:
                    nn.init.uniform_(param.data)

def init_weights_kaiming_uniform(module):
    for m in module.modules():
        if isinstance(m, nn.Conv1d):
            nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Conv2d):
            nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Conv3d):
            nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm1d):
            nn.init.uniform_(m.weight, a=0, b=1)
            nn.init.constant_(m.bias, val=0.)
        elif isinstance(m, nn.BatchNorm2d):
            nn.init.uniform_(m.weight, a=0, b=1)
            nn.init.constant_(m.bias, val=0.)
        elif isinstance(m, nn.BatchNorm3d):
            nn.init.uniform_(m.weight, a=0, b=1)
            nn.init.constant_(m.bias, val=0.)
        elif isinstance(m, nn.Linear):
            nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, val=0.)

def init_weights_kaiming_normal(module):
    for m in module.modules():
        if isinstance(m, nn.Conv1d):
            nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        if isinstance(m, nn.Conv3d):
            nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm1d):
            nn.init.normal_(m.weight, 0, 0.01)
            nn.init.constant_(m.bias, val=0.)
        elif isinstance(m, nn.BatchNorm2d):
            nn.init.normal_(m.weight, 0, 0.01)
            nn.init.constant_(m.bias, val=0.)
        elif isinstance(m, nn.BatchNorm3d):
            nn.init.normal_(m.weight, 0, 0.01)
            nn.init.constant_(m.bias, val=0.)
        elif isinstance(m, nn.Linear):
            nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, val=0.)

### Linear FC Blocks
def linear_block(n_inputs_loc, hidden_loc,
                 activ_config=None,batch_config=None,p_drop_loc=0.1):

    # Dictionary defining Block Architecture
    BlockArchitecture=[]

    hidden_loc.insert(0,n_inputs_loc)

    if activ_config==None:
        activ_config=repmat('no_activ',len(hidden_loc),1)
    if batch_config==None:
        batch_config=repmat('no_batch',len(hidden_loc),1)
    #Block Layers List
    for i in np.arange(len(hidden_loc)-1):
        BlockArchitecture.append(('linear'+str(i+1),
                                  nn.Linear(hidden_loc[i], hidden_loc[i+1])))

        if(activ_config[i]=='relu'):
            BlockArchitecture.append(('relu'+str(i+1),nn.ReLU(inplace=True)))

        elif(activ_config[i]=='tanh'):
            BlockArchitecture.append(('tanh'+str(i+1),nn.Tanh()))
        elif(activ_config[i]=='relu6'):
             BlockArchitecture.append(('relu6'+str(i+1),nn.ReLU6(inplace=True)))

        if(batch_config[i]=='batch'):
            BlockArchitecture.append(('batch'+str(i+1),nn.BatchNorm1d( hidden_loc[i+1])))

        BlockArchitecture.append(('drop'+str(i+1),nn.Dropout(p_drop_loc)))
    linear_block_loc = nn.Sequential(
        OrderedDict(BlockArchitecture)
        )
    return linear_block_loc


class LinearBlock(nn.Module):
    """
    MultiLayer Perceptron:
    Netwotk with n_hidden layers with architecture linear+drop+relu+batch
     Constructor Parameters:
           n_inputs: dimensionality of input features (n_channels * n_features , by default)
                     n_channels (=14), number of sensors or images for each case
                     n_features(=40), number of features for each n_channels
           n_classes: number of output classes (=3, by default)
           hidden(=[128,128], default): list with the number of neurons for each hidden layer
           p_drop(=0.1, default): probability for Drop layer (=0, no drop is performed)

    """

    def __init__(self, inputmodule_params,net_params):
        super().__init__()



        ### Input Parameters
        self.n_inputs = inputmodule_params['n_inputs']


        self.hidden=net_params['hidden']
        self.dropout=net_params['dropout']
        if net_params['dropout'] is None:
            self.dropout=0.5
        self.nlayers=len(self.hidden)
        if 'activ_config' not in list(net_params.keys()):

            self.activ_config=None
        else:
             self.activ_config=net_params['activ_config']

        if 'batch_config' not in list(net_params.keys()):
            self.batch_config=None
        else:
            self.batch_config=net_params['batch_config']



        self.linear_block0= linear_block(self.n_inputs, self.hidden.copy(),
                                                 activ_config=self.activ_config,
                                                 batch_config=self.batch_config,
                                                 p_drop_loc=self.dropout)



      #  self.fc_out=nn.Identity()
        # weight init
        init_weights_xavier_normal(self)

    def forward(self, x):


        return self.linear_block0(x)

### Convolutional
class _CNNLayer(nn.Module):
    def __init__(
        self, num_input_features: int, n_neurons: int, kernel_sze:int =3,
        stride:int=1,
        drop_rate: float=0,
        Relu=True
         ) -> None:
        super().__init__()


        norm1 = nn.BatchNorm2d(n_neurons)
        conv1 = nn.Conv2d(num_input_features, n_neurons, kernel_size=kernel_sze,
                               stride=stride, padding=(int((kernel_sze-1)/2)))

      #  relu1 = nn.ReLU(inplace=True)
        relu1= nn.LeakyReLU(inplace=True)

        drop=nn.Dropout(drop_rate)
        if Relu:
            self.cnn_layer=nn.Sequential(conv1,norm1,relu1,drop)
        else:
            self.cnn_layer=nn.Sequential(conv1,norm1,drop)
        init_weights_xavier_normal(self)

    def forward(self, x: Tensor):

        return(self.cnn_layer(x))

class _UnCNNLayer(nn.Module):
    def __init__(
        self, num_input_features: int, n_neurons: int, kernel_sze:int =3,
        stride:int=2,
        drop_rate: float=0,
        Relu=True
         ) -> None:
        super().__init__()

        self.stride=stride
        norm1 = nn.BatchNorm2d(n_neurons)
        conv1 = nn.ConvTranspose2d(num_input_features, n_neurons, kernel_size=kernel_sze,
                               stride=stride, padding=(int((kernel_sze-1)/2)))


     #   relu1 = nn.ReLU(inplace=True)
        relu1 = nn.LeakyReLU(inplace=True)

        drop=nn.Dropout(drop_rate)
        if Relu:
            self.cnn_layer=nn.Sequential(conv1,norm1,relu1,drop)
        else:
            self.cnn_layer=nn.Sequential(conv1,norm1,drop)
        init_weights_xavier_normal(self)

    def forward(self, x: Tensor):

        if  self.stride>1:
            sze_enc=x.shape[-1]
            x=self.cnn_layer[0](x,output_size=(sze_enc*2,sze_enc*2))
            for k in np.arange(1,len(self.cnn_layer)):
                x=self.cnn_layer[k](x)
        else:
            x=self.cnn_layer(x)

        return(x)

    # def forward(self, x1, x2):
    #     x1 = self.cnn_layer(x1)
    #     # input is CHW
    #     diffY = x2.size()[2] - x1.size()[2]
    #     diffX = x2.size()[3] - x1.size()[3]

    #     x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
    #                     diffY // 2, diffY - diffY // 2])
    #     # if you have padding issues, see
    #     # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
    #     # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
    #     x = torch.cat([x2, x1], dim=1)
    #     return self.conv(x)

class _CNNBlock(nn.ModuleDict):
    _version = 2

    def __init__(
        self,
        num_input_channels: int=1,
        drop_rate=0,
        block_config = (64,128),
        stride=None,
        decoder=False,
        Relu=True

    ) -> None:
        super().__init__()

        num_layers=len(block_config)
        self.num_input_channels=num_input_channels
        print('block inp ch',num_input_channels)

        if stride is None:
            stride=np.ones(num_layers)

        for i in range(num_layers):
            if decoder==True:
                layer = _UnCNNLayer(
                    num_input_channels,
                    n_neurons=block_config[i],
                    stride=stride[i],
                    drop_rate=drop_rate

                )
            else:
                layer = _CNNLayer(
                    num_input_channels,
                    n_neurons=block_config[i],
                    stride=stride[i],
                    drop_rate=drop_rate,
                    Relu=Relu

                )
            self.add_module("cnnlayer%d" % (i + 1), layer)
            num_input_channels=block_config[i]

    def forward(self, x: Tensor) -> Tensor:

        for name, layer in self.items():
            x = layer(x)


        return x



In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Encoder(nn.Module):
    r"""Encoder class
    `".
    Input Parameters:
        1. inputmodule_params: dictionary with keys ['num_input_channels']
            inputmodule_params['num_input_channels']=Channels of input images
        2. net_params: dictionary defining architecture:
            net_params['block_configs']: list of number of neurons for each
            convolutional block. A block can have more than one layer
            net_params['stride']:list of strides for each block layers
            net_params['drop_rate']: value of the Dropout (equal for all blocks)
        Examples:
            1. Encoder with 4 blocks with one layer each
            net_params['block_configs']=[[32],[64],[128],[256]]
            net_params['stride']=[[2],[2],[2],[2]]
            2. Encoder with 2 blocks with two layers each
            net_params['block_configs']=[[32,32],[64,64]]
            net_params['stride']=[[1,2],[1,2]]

    """

    def __init__(self, inputmodule_params,net_params):
        super().__init__()


        num_input_channels=inputmodule_params['num_input_channels']



        drop_rate=net_params['drop_rate']
        block_configs=net_params['block_configs'].copy()
        n_blocks=len(block_configs)
        if 'stride' in net_params.keys():
            stride=net_params['stride']
        else:
            stride=[]
            for i in np.arange(len(block_configs)):
                stride.append(list(np.ones(len(block_configs[i])-1,dtype=int))+[2])

        # Encoder
        self.encoder=nn.Sequential(
            )
        outchannels_encoder=[]
        for i in np.arange(n_blocks):
            print('block',i)
            block = _CNNBlock(
                num_input_channels=num_input_channels,
                drop_rate=drop_rate,
                block_config=block_configs[i],
                stride= stride[i]

            )
            self.encoder.add_module("cnnblock%d" % (i + 1), block)

            if stride==1:
                self.encoder.add_module("mxpool%d" % (i + 1),
                                         nn.MaxPool2d(kernel_size=2, stride=2, padding=0))

            num_input_channels=block_configs[i][-1]
           # outchannels_encoder.append(num_input_channels)



    def forward(self, x: Tensor) -> Tensor:

        x=self.encoder(x)

        return x

class Decoder(nn.Module):
    r"""Decoder class
    `".
    Input Parameters:
        1. inputmodule_params: dictionary with keys ['num_input_channels']
            inputmodule_params['num_input_channels']=Channels of input images
        2. net_params: dictionary defining architecture:
            net_params['block_configs']: list of number of neurons for each conv block
            net_params['stride']:list of strides for each block layers
            net_params['drop_rate']: value of the Dropout (equal for all blocks)
    """
    def __init__(self, inputmodule_params,net_params):
        super().__init__()


        num_input_channels=inputmodule_params['num_input_channels']

        self.upPoolMode='bilinear'


        drop_rate=net_params['drop_rate']
        block_configs=net_params['block_configs'].copy()
        self.n_blocks=len(block_configs)

        if 'stride' in net_params.keys():
            stride=net_params['stride']
        else:
            stride=[]
            for i in np.arange(len(block_configs)):
                stride.append(list(np.ones(len(block_configs[i])-1,dtype=int))+[2])


        # Decoder
        self.decoder=nn.Sequential(
            )

        for i0 in np.arange(self.n_blocks)[::-1]:
            i=self.n_blocks-(i0+1)
            block = _CNNBlock(
                num_input_channels=num_input_channels,
                drop_rate=drop_rate,
                block_config=block_configs[i],
                stride=stride[i],
                decoder=True
            )

            # if stride==1:
            #     self.decoder.add_module("uppool%d" % (i + 1),
            #                               nn.Upsample(scale_factor=2,
            #                                           mode=self.upPoolMode, align_corners=True))

            self.decoder.add_module("cnnblock%d" % (i0+1), block)


            num_input_channels=block_configs[i][-1]


        self.decoder[-1][list(self.decoder[-1].keys())[-1]].cnn_layer[2]=nn.Identity()

    def forward(self, x: Tensor) -> Tensor:

        input_sze=x.shape

     #   for i in np.arange(n_blocks)[::-1]:

        x=self.decoder(x)

        return x

##### GENERATIVE MODELS
class AutoEncoderCNN(nn.Module):
    r"""AutoEncoderCNN model class
    `".
    Input Parameters:
        1. inputmodule_paramsEnc: dictionary with keys ['num_input_channels']
            inputmodule_paramsEnc['num_input_channels']=Channels of input images
        2. net_paramsEnc: dictionary defining architecture of the Encoder (see Encoder class)
        3. inputmodule_paramsDec: dictionary with keys ['num_input_channels']
           inputmodule_paramsDec['num_input_channels']=Channels of input images
        4. net_paramsDec: dictionary defining architecture of the Encoder (see Decoder/Encoder classes)
    """

    def __init__(self, inputmodule_paramsEnc,net_paramsEnc,inputmodule_paramsDec,net_paramsDec):
        super().__init__()

        self.inputmodule_paramsEnc=inputmodule_paramsEnc
        self.inputmodule_paramsDec=inputmodule_paramsDec
        self.net_paramsEnc=net_paramsEnc
        self.net_paramsDec=net_paramsDec

        # Encoder
        self.encoder=Encoder(inputmodule_paramsEnc,net_paramsEnc)

        # Decoder
        self.decoder=Decoder(inputmodule_paramsDec,net_paramsDec)

    def forward(self, x: Tensor) -> Tensor:
            # Guardamos el tamaño de la imagen de entrada para usarlo al final (en la fase de upsampling)
            input_size = x.shape

            # Paso por el Encoder
            encoded = self.encoder(x)

            # Paso por el Decoder
            decoded = self.decoder(encoded)

            return decoded

In [13]:
def AEConfigs(Config):

    if Config=='1':
        # CONFIG1
        net_paramsEnc['block_configs']=[[32,32],[64,64]]
        net_paramsEnc['stride']=[[1,2],[1,2]]
        net_paramsEnc['drop_rate']=0.1
        net_paramsDec['drop_rate']=0.1
        net_paramsDec['block_configs']=[[64,32],[32,inputmodule_paramsEnc['num_input_channels']]]
        net_paramsDec['stride']=net_paramsEnc['stride']
        inputmodule_paramsDec['num_input_channels']=net_paramsEnc['block_configs'][-1][-1]



    elif Config=='2':
        # CONFIG 2
        net_paramsEnc['block_configs']=[[32],[64],[128],[256]]
        net_paramsEnc['stride']=[[2],[2],[2],[2]]
        net_paramsDec['block_configs']=[[128],[64],[32],[inputmodule_paramsEnc['num_input_channels']]]
        net_paramsDec['stride']=net_paramsEnc['stride']
        inputmodule_paramsDec['num_input_channels']=net_paramsEnc['block_configs'][-1][-1]


    elif Config=='3':
        # CONFIG3
        net_paramsEnc['block_configs']=[[32],[64],[64]]
        net_paramsEnc['stride']=[[1],[2],[2]]
        net_paramsDec['block_configs']=[[64],[32],[inputmodule_paramsEnc['num_input_channels']]]
        net_paramsDec['stride']=net_paramsEnc['stride']
        inputmodule_paramsDec['num_input_channels']=net_paramsEnc['block_configs'][-1][-1]

    return net_paramsEnc,net_paramsDec,inputmodule_paramsDec


######################### 0. EXPERIMENT PARAMETERS

# 0.2 Parámetros y definiciones de red
inputmodule_paramsEnc = {'num_input_channels': 3}
net_paramsEnc = {}
net_paramsDec = {}
inputmodule_paramsDec = {}

# 0.1 AE PARAMETERS
inputmodule_paramsEnc={}
inputmodule_paramsEnc['num_input_channels']=3

###### CONFIG1
Config='1'
net_paramsEnc,net_paramsDec,inputmodule_paramsDec=AEConfigs(Config)

In [14]:
# load model
model = AutoEncoderCNN(inputmodule_paramsEnc, net_paramsEnc,
                     inputmodule_paramsDec, net_paramsDec)
model.load_state_dict(torch.load('best_model_AECNN3_50.pth'))


block 0
block inp ch 3
block 1
block inp ch 32
block inp ch 64
block inp ch 32


<All keys matched successfully>

In [15]:
print(model)

AutoEncoderCNN(
  (encoder): Encoder(
    (encoder): Sequential(
      (cnnblock1): _CNNBlock(
        (cnnlayer1): _CNNLayer(
          (cnn_layer): Sequential(
            (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): LeakyReLU(negative_slope=0.01, inplace=True)
            (3): Dropout(p=0.1, inplace=False)
          )
        )
        (cnnlayer2): _CNNLayer(
          (cnn_layer): Sequential(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): LeakyReLU(negative_slope=0.01, inplace=True)
            (3): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (cnnblock2): _CNNBlock(
        (cnnlayer1): _CNNLayer(
          (cnn_layer): Sequential(
            (0): Conv2d(32, 64, kernel_size=(

In [16]:
import pandas as pd
import numpy as np
import cv2
import os
import glob
import random
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data


def load_cropped(folder_path, csv_path, patient_list = [], sample_size=200, prints = False):
    # Cargar el CSV con pandas
    df = pd.read_csv(csv_path)

    # Convertir el CSV a un diccionario para un acceso rápido
    patient_metadata = {row['CODI']: row['DENSITAT'] for _, row in df.iterrows()}

    # Inicializar la estructura de datos para almacenar los datos de los pacientes seleccionados
    patients_data = []
    images_list = []
    patients_list_img = []

    # si no se proporciona una lista de pacientes, se seleccionan todos los pacientes iterando folder_path
    if len(patient_list) == 0:
        for patient_folder in glob.glob(os.path.join(folder_path, "*")):
            patient_id = os.path.basename(patient_folder).split("_")[0]
            patient_list.append(patient_id)

    incorrect_shape = 0
    # Iterar sobre cada paciente en la lista de IDs proporcionada
    for i,patient_id in enumerate(patient_list):
        # Obtener carpeta del paciente
        try:
            patient_folder = glob.glob(os.path.join(folder_path, f"{patient_id}_*"))[0]
        except:
            continue

        # Verificar que el paciente esté en el CSV
        if patient_id in patient_metadata:
            # Obtener todas las imágenes .png dentro de la carpeta del paciente
            images = glob.glob(os.path.join(patient_folder, "*.png"))

            # Si el paciente tiene imágenes en su carpeta
            if images:
                # Mezclar la lista de imágenes
                random.shuffle(images)

                if sample_size == -1:
                    sample_size = len(images)
                # Seleccionar una muestra de tamaño sample_size o menos si hay menos imágenes
                images_sampled = random.sample(images, min(sample_size, len(images)))

                for j,image_path in enumerate(images_sampled):
                    if j % 200 == 0:
                        if prints:
                            print(f"Processing patient {i+1}/{len(patient_list)} - image {j+1}/{len(images_sampled)}")

                    # Cargar la imagen en formato BGR con cv2
                    image_bgr = cv2.imread(image_path)

                    # Convertir la imagen de BGR a RGB
                    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

                    if image_rgb.shape[0] != 256 or image_rgb.shape[1] != 256:
                        incorrect_shape +=1
                        # Resize de la imagen a 256x256
                        image_rgb = cv2.resize(image_rgb, (256, 256))

                    # Permutar canales
                    image_rgb = np.transpose(image_rgb, (2, 0, 1))

                    # Pasar de uint8 a float32
                    image_rgb = image_rgb/255.0

                    # Añadir la imagen a la lista de imágenes en formato RGB
                    images_list.append(image_rgb)
                # Binariar densidad
                if patient_metadata[patient_id] == "NEGATIVA":
                    dens = 0
                else:
                    dens = 1

                # Añadir la densidad a la lista de metadatos
                patients_data.extend([dens] * len(images_sampled))
                patients_list_img.extend([patient_id] * len(images_sampled))

    print(f'Resized images: {incorrect_shape}/{len(images_list)}')

    return images_list, patients_data, patients_list_img

# Paso 2: Crear la clase Standard_Dataset
class Standard_Dataset(data.Dataset):
    def __init__(self, X, Y=None, transformation=None):
        super().__init__()
        self.X = X
        self.y = Y
        self.transformation = transformation

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = self.X[idx]
        if self.transformation:
            image = self.transformation(image)

        image_tensor = torch.from_numpy(image).float()

        if self.y is not None:
            label = torch.tensor(self.y[idx], dtype=torch.float32)
            return image_tensor, label
        else:
            return image_tensor

# Paso 3: Cargar los datos
folder_path = "HoldOut"
csv_path = "PatientDiagnosis.csv"
patient_list = []
sample_size = 1

# Filtrando imágenes y etiquetas para obtener solo aquellas cuya densidad es 0
images, labels, pat_list_img = load_cropped(folder_path, csv_path, patient_list, sample_size)
dataset_holdour = Standard_Dataset(images)
dataloader_holder = DataLoader(dataset_holdour, batch_size=1, shuffle=False)

Resized images: 0/116


In [ ]:

dataset_holdour = Standard_Dataset(images)
dataloader_holder = DataLoader(dataset_holdour, batch_size=1, shuffle=False)


In [19]:
fred_values = get_fred(model, dataloader_holder, device="cpu")
print('len fred',len(fred_values))
print(fred_values)

len fred 116
[2.0, 2.0, 0.0, 9.0, 27.0, 64.0, 0.0, 37.0, 102.0, 31.0, 0.0, 579.0, 94.0, 66.0, 116.0, 506.0, 538.0, 696.0, 8.0, 21.0, 1720.0, 6.0, 0.0, 0.0, 4.0, 150.0, 0.0, 49.0, 0.0, 0.0, 98.0, 247.0, 109.0, 21.0, 160.0, 12.0, 1.0, 5.0, 95.0, 0.0, 44.0, 47.0, 0.0, 0.0, 5.0, 0.0, 2.0, 4.0, 109.0, 1.0, 0.0, 2.0, 0.0, 87.0, 1.0, 0.0, 2648.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 1.0, 0.0, 0.0, 14.0, 0.0, 0.0, 0.0, 1.0, 4.0, 48.0, 0.0, 13.0, 4.0, 0.0, 8.0, 1.0, 25.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 429.0, 146.0, 4.0, 487.0, 17.0, 1.0, 2.0, 295.0, 0.0, 4.0, 101.0, 371.0, 2.0, 4974.0, 10.0, 16.0, 6.0, 18.0, 0.0, 0.0, 12.0, 3012.0, 0.0, 46.0, 0.0, 0.0, 2.0, 34.0]


In [25]:
def classify_FRED_threshold(fred_values, patch_threshold_fred):
    img_positive = np.array(fred_values) >= patch_threshold_fred
    return img_positive

def classify_PATIENT_threshold(fred_values, patient_val, patient_image_list, patch_threshold_fred, patient_threshold_fred):
    img_positive = classify_FRED_threshold(fred_values, patch_threshold_fred)
    unique_patients = np.unique(patient_val)
    patient_positive_counts = []

    for patient_id in unique_patients:
        patient_indices = np.where(np.array(patient_image_list) == patient_id)[0]
        patient_positive_counts.append(np.sum(img_positive[patient_indices])/ len(patient_indices))

    patient_positive = np.array(patient_positive_counts) >= patient_threshold_fred

    return patient_positive

In [26]:
import pandas as pd

# Cargar el CSV
csv_path = "patches_thresholds.csv"  # Asegúrate de que el nombre del archivo sea correcto
data = pd.read_csv(csv_path)

# Calcular la media de la columna 'threshold'
patches_mean_threshold = data['threshold'].mean()

print(f"La media de los valores en la columna 'threshold' es: {patches_mean_threshold}")

# Cargar el CSV
csv_path = "patient_thresholds.csv"  # Asegúrate de que el nombre del archivo sea correcto
data = pd.read_csv(csv_path)

# Calcular la media de la columna 'threshold'
patient_mean_threshold = data['threshold'].mean()

print(f"La media de los valores en la columna 'threshold' es: {patient_mean_threshold}")


La media de los valores en la columna 'threshold' es: 107.4
La media de los valores en la columna 'threshold' es: 0.20999999999999996


In [29]:
patient_positive = classify_PATIENT_threshold(fred_values, images, pat_list_img, patches_mean_threshold, patient_mean_threshold)
print(patient_positive)

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False Fa

C:\Users\marbj\AppData\Local\Temp\ipykernel_8716\679301622.py:11: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  patient_indices = np.where(np.array(patient_image_list) == patient_id)[0]
C:\Users\marbj\AppData\Local\Temp\ipykernel_8716\679301622.py:12: RuntimeWarning: invalid value encountered in scalar divide
  patient_positive_counts.append(np.sum(img_positive[patient_indices])/ len(patient_indices))


In [38]:
import pandas as pd
import numpy as np


def calcular_media(data):
    return data['threshold'].mean()

def calcular_mediana(data):
    return data['threshold'].median()

def calcular_desviacion_estandar(data):
    return data['threshold'].std()

def calcular_minimo(data):
    return data['threshold'].min()

def calcular_maximo(data):
    return data['threshold'].max()

def calcular_cuartiles(data):
    q1 = data['threshold'].quantile(0.25)
    q3 = data['threshold'].quantile(0.75)
    return q1, q3

def calcular_varianza(data):
    return data['threshold'].var()

def resumen_descriptivo(data):
    return data['threshold'].describe()

def normalizar(data):
    min_val = data['threshold'].min()
    max_val = data['threshold'].max()
    data['normalized_threshold'] = (data['threshold'] - min_val) / (max_val - min_val)
    return data

def calcular_embedding(data):
    return np.mean(data['threshold'].values)

In [39]:
data = pd.read_csv('patches_thresholds.csv')

media = calcular_media(data)
mediana = calcular_mediana(data)
std_dev = calcular_desviacion_estandar(data)
q1, q3 = calcular_cuartiles(data)
varianza = calcular_varianza(data)
normalizado = normalizar(data)
embedding = calcular_embedding(data)
minimo = calcular_minimo(data)
maximo = calcular_maximo(data)

print(f"Media: {media}")
print(f"Mediana: {mediana}")
print(f"Minimo: {minimo}")
print(f"Maximo: {maximo}")
#print(f"Desviación estándar: {std_dev}")
print(f"Cuartiles: Q1={q1}, Q3={q3}")
#print(f"Varianza: {varianza}")
#print("Datos normalizados:")
#print(normalizado)
print(embedding)


Media: 107.4
Mediana: 107.0
Minimo: 101
Maximo: 113
Cuartiles: Q1=104.0, Q3=112.0
107.4


In [56]:
import pandas as pd

# Cargar el archivo CSV
csv_path = "metrics_th2.csv"  # Cambia al nombre de tu archivo
data = pd.read_csv(csv_path, delimiter=",")  # Usa `delimiter="\t"` si es TSV


# Calcular el rango (mínimo y máximo) para cada columna de métricas
def calcular_rangos(data):
    rangos = {}
    #print(data)
    #print(data.columns)
    for columna in data.columns:  # Excluir la columna `fold`
        minimo = data[columna].min()
        maximo = data[columna].max()
        #print(minimo, maximo)
        rangos[columna] = (minimo, maximo)
    return rangos

# Llamar a la función y mostrar resultados
rangos = calcular_rangos(data)
#print(rangos)

# Imprimir los rangos
for metrica, (min_val, max_val) in rangos.items():
    print(f"Métrica: {metrica}")
    print(f"  Mínimo: {min_val}")
    print(f"  Máximo: {max_val}")
    print("---")

Métrica: fold
  Mínimo: 0
  Máximo: 4
---
Métrica: accuracy
  Mínimo: 0.2903225806451613
  Máximo: 0.7096774193548387
---
Métrica: precision
  Mínimo: 0.3
  Máximo: 0.7333333333333333
---
Métrica: recall
  Mínimo: 0.3333333333333333
  Máximo: 0.6875
---
Métrica: f1_score
  Mínimo: 0.3157894736842105
  Máximo: 0.7096774193548386
---
Métrica: tp
  Mínimo: 3
  Máximo: 11
---
Métrica: tn
  Mínimo: 3
  Máximo: 15
---
Métrica: fp
  Mínimo: 4
  Máximo: 12
---
Métrica: fn
  Mínimo: 5
  Máximo: 10
---
